**Artificial Neural Network - Churn Modelling for Bank Customers**

In [34]:
import numpy as np
import pandas as pd
import tensorflow as tf
import joblib

In [35]:
tf.__version__

'2.10.0'

**Data Preprocessing**

In [36]:
dataset = pd.read_csv('../ann-churn-modelling/resources/Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [37]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [38]:
print(y)

[1 0 1 ... 1 1 0]


**Encode Categorical Data**

In [39]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2]) #label encode gender column
# Saving the LabelEncoder
joblib.dump(le, 'label_encoder.joblib')

['label_encoder.joblib']

In [40]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encode Geo Col

In [41]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

#Save encoder so we can load it later in the script
encoder = ct.named_transformers_['encoder']
joblib.dump(encoder, 'onehot_encoder.joblib')

['onehot_encoder.joblib']

In [42]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [43]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

**Feature Scaling**

In [44]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

**Building the ANN**

In [45]:
ann = tf.keras.models.Sequential()

In [46]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu')) # add first hidden layer with 6 neurons and the rectifier activation function

In [47]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [48]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

**Compiling the ANN**

In [49]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [50]:
ann.fit(X_train, y_train, batch_size = 32, epochs=100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.6206 - accuracy: 0.6867
Epoch 2/100
250/250 [==============================] - 0s 997us/step - loss: 0.4600 - accuracy: 0.8009
Epoch 3/100
250/250 [==============================] - 0s 951us/step - loss: 0.4376 - accuracy: 0.8055
Epoch 4/100
250/250 [==============================] - 0s 994us/step - loss: 0.4279 - accuracy: 0.8065
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4218 - accuracy: 0.8080
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4173 - accuracy: 0.8077
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4131 - accuracy: 0.8090
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4090 - accuracy: 0.8086
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4042 - accuracy: 0.8065
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3976 - 

**Make a single prediction**

In [51]:
print(ann.predict(sc.transform([[1.0, 0.0, 0.0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 72ms/step
[[False]]


Need to sort out how to one hot encode and label encode a single input of data...

In [52]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), y_test.reshape(len(y_test), 1)),1))

63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [53]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1513   82]
 [ 195  210]]


0.8615

**Scripts to use in Main class for loading model**

In [102]:
X_new = np.array([[600, 'France', 'Male', 40, 3, 60000, 2, 1, 1, 50000]])
saved_labelencoder = joblib.load('label_encoder.joblib')
X_new[:, 2] = saved_labelencoder.transform(X_new[:, 2]) #label encode gender column
print(X_new)

[['600' 'France' '1' '40' '3' '60000' '2' '1' '1' '50000']]


In [103]:
print(X_new)


[['600' 'France' '1' '40' '3' '60000' '2' '1' '1' '50000']]


In [104]:
from sklearn.preprocessing import OneHotEncoder
saved_encoder = joblib.load('onehot_encoder.joblib')
encoded_values = saved_encoder.transform(X_new[:, [1]]).toarray()
X_new = np.delete(X_new, 1, axis=1) #remove nonencoded country column
X_new = np.hstack((encoded_values, X_new)) #concatenate because OneHotEncoder pushes encoded values to beginning of array

[[1. 0. 0.]]


In [105]:
print(X_new)

[['1.0' '0.0' '0.0' '600' '1' '40' '3' '60000' '2' '1' '1' '50000']]


In [61]:
# from sklearn.preprocessing import OneHotEncoder
# categories = [np.array(['France', 'USA', 'Germany'])]
# encoder = OneHotEncoder(categories=categories)
# transformers = [('one_hot_encode', encoder, [1])]  # Applying encoding to column 0
# # Create a ColumnTransformer
# ct_new = ColumnTransformer(transformers, remainder='passthrough')
# X_new = ct_new.fit_transform(X_new)
# print(X_new)

[['1.0' '0.0' '0.0' '600' '1' '40' '3' '60000' '2' '1' '1' '50000']]


In [106]:
print(ann.predict(sc.transform(X_new)))

1/1 [==============================] - 0s 22ms/step
[[0.01970788]]
